# 0. Data Inspection: this dataset is problematic

Largest issues:
- no customer id in Table ListOfOrders, only customer first name + last name. So we cannot for sure connect customers to orders
- no primary keys in the tables
- customer id in CustomerList1 is a real value, but customer is in CustomerList2 is an integer
- 7 missing values for customer id in table CustomerList1
- order items also don't have an id, just a name of the product
- dataset is limited and there is no guarantee that these are all orders
- dataset doesn't contain a margin field, so we cannot say much about profitability

*In essence we can only do some exploration, but no real conclusions on profitability etc.*

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
sqlite_connection = sqlite3.connect('assessment_database.sqlite')

In [3]:
tables_query = """
    SELECT name 
    FROM sqlite_master 
    WHERE type='table';
"""

df_tables = pd.read_sql(tables_query, sqlite_connection)
df_tables

,name
0,CustomerList1
1,CustomerList2
2,ListOfOrders
3,OrderBreakdown
4,customers_cleaned


In [4]:
# no primary keys!!

# Customer_id in table1 has datatype REAL and Customer_id in table2 has datatype INTEGER

# ListOfOrders: No CustomerId in table ListOfOrders ?! Only Customer Name

# OrderBreakdown: Total revenue before discount is a text field?!

for table_name in df_tables['name']:
    table_info_query = f"""PRAGMA table_info({table_name})"""
    table_info = pd.read_sql(table_info_query, sqlite_connection)
    print(table_name.upper(), table_info, sep='\n', end='\n\n')

CUSTOMERLIST1
   cid                    name       type  notnull dflt_value  pk
0    0                   index    INTEGER        0       None   0
1    1             Customer_id       REAL        0       None   0
2    2                    Name       TEXT        0       None   0
3    3                 Surname       TEXT        0       None   0
4    4                 country       TEXT        0       None   0
5    5                  Gender       TEXT        0       None   0
6    6                     Age       REAL        0       None   0
7    7  Date Registered Online  TIMESTAMP        0       None   0

CUSTOMERLIST2
   cid                    name       type  notnull dflt_value  pk
0    0                   index    INTEGER        0       None   0
1    1             Customer_id    INTEGER        0       None   0
2    2                    Name       TEXT        0       None   0
3    3                 Surname       TEXT        0       None   0
4    4                 country       TEXT      

In [5]:
# 749 customer rows in list 1

# need to convert customer_id to int 

customer_list1_query = """SELECT * FROM CustomerList1"""

df_customer_list1 = pd.read_sql(customer_list1_query, sqlite_connection, index_col='index')
df_customer_list1

,Customer_id,Name,Surname,country,Gender,Age,Date Registered Online
index,,,,,,,
0,350714120.0,Aaron,Bergman,United Kingdom,male,48.0,2011-01-17 00:00:00
1,370582569.0,Aaron,Hawkins,United Kingdom,male,64.0,2011-05-31 00:00:00
2,229469415.0,Aaron,Smayling,United Kingdom,male,29.0,2011-02-17 00:00:00
3,211644956.0,Adam,Bellavance,United Kingdom,male,41.0,2012-12-08 00:00:00
4,131647193.0,Adam,Hart,United Kingdom,male,30.0,2011-04-11 00:00:00
...,...,...,...,...,...,...,...
744,272070547.0,Thomas,Thornton,Spain,male,55.0,2011-09-18 00:00:00
745,334707196.0,Tiffany,House,Switzerland,female,49.0,2011-07-20 00:00:00
746,269572786.0,Tim,Brockman,Spain,male,53.0,2011-08-25 00:00:00


In [6]:
df_customer_list1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 749 entries, 0 to 748
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer_id             742 non-null    float64
 1   Name                    688 non-null    object 
 2   Surname                 688 non-null    object 
 3   country                 743 non-null    object 
 4   Gender                  721 non-null    object 
 5   Age                     722 non-null    float64
 6   Date Registered Online  741 non-null    object 
dtypes: float64(2), object(5)
memory usage: 46.8+ KB


In [7]:
# 7 missing values for customer id

df_customer_list1.isna().sum()

Customer_id                7
Name                      61
Surname                   61
country                    6
Gender                    28
Age                       27
Date Registered Online     8
dtype: int64

In [8]:
df_customer_list1.nunique()

Customer_id               742
Name                      313
Surname                   532
country                    12
Gender                      3
Age                        59
Date Registered Online    490
dtype: int64

In [9]:
# rows with missing values

df_customer_list1[df_customer_list1.isna().sum(axis=1) >= 1]

,Customer_id,Name,Surname,country,Gender,Age,Date Registered Online
index,,,,,,,
10,301686124.0,None,None,United Kingdom,None,NaN,2010-12-20 00:00:00
21,321069499.0,None,None,United Kingdom,None,NaN,2011-05-22 00:00:00
44,361656560.0,None,None,United Kingdom,None,NaN,2010-12-16 00:00:00
45,133222542.0,None,None,United Kingdom,None,NaN,2011-07-24 00:00:00
57,299147547.0,None,None,Germany,None,NaN,None
...,...,...,...,...,...,...,...
688,194251595.0,None,None,United Kingdom,male,58.0,2011-08-26 00:00:00
701,345994702.0,None,None,United Kingdom,female,52.0,2011-09-20 00:00:00
716,139007107.0,Stuart,None,Switzerland,male,53.0,2011-01-13 00:00:00


In [10]:
# customers without id

df_customer_list1[df_customer_list1['Customer_id'].isna()]

,Customer_id,Name,Surname,country,Gender,Age,Date Registered Online
index,,,,,,,
385,NaN,John,Huston,United Kingdom,male,38.0,2011-03-30 00:00:00
432,NaN,None,None,Italy,female,41.0,2011-03-09 00:00:00
488,NaN,None,None,None,female,53.0,2012-09-22 00:00:00
532,NaN,Michelle,Lonsdale,United Kingdom,female,41.0,2011-05-09 00:00:00
559,NaN,Nathan,Mautz,United Kingdom,male,48.0,2013-06-01 00:00:00
584,NaN,Patrick,Gardner,Germany,male,48.0,2011-12-06 00:00:00
669,NaN,None,None,United Kingdom,male,56.0,2011-12-12 00:00:00


In [11]:
# max age = 999, min age = 0, and also missing age values 

df_customer_list1.describe(include='all')

,Customer_id,Name,Surname,country,Gender,Age,Date Registered Online
count,7.420000e+02,688,688,743,721,722.000000,741
unique,NaN,313,532,12,3,NaN,490
top,NaN,Michael,Jones,United Kingdom,male,NaN,2011-06-13 00:00:00
freq,NaN,10,5,446,438,NaN,7
mean,2.486832e+08,NaN,NaN,NaN,NaN,50.121884,NaN
std,8.490759e+07,NaN,NaN,NaN,NaN,36.845542,NaN
min,1.002696e+08,NaN,NaN,NaN,NaN,0.000000,NaN
25%,1.771247e+08,NaN,NaN,NaN,NaN,42.250000,NaN
50%,2.459066e+08,NaN,NaN,NaN,NaN,50.000000,NaN
75%,3.234661e+08,NaN,NaN,NaN,NaN,55.000000,NaN


In [12]:
# 693 customer rows in list 2

customer_list2_query = """SELECT * FROM CustomerList2"""

df_customer_list2 = pd.read_sql(customer_list2_query, sqlite_connection, index_col='index')
df_customer_list2

,Customer_id,Name,Surname,country,Gender,Age,Date Registered Online
index,,,,,,,
0,334708301,Bradley,Nguyen,United Kingdom,male,50.0,2012-09-05 00:00:00
1,389480478,Bradley,Talbott,United Kingdom,male,61.0,2012-08-02 00:00:00
2,330461776,Brenda,Bowman,Portugal,male,37.0,2012-05-27 00:00:00
3,243694808,Brendan,Dodson,United Kingdom,male,33.0,2011-05-13 00:00:00
4,386215654,Brendan,Murry,France,male,47.0,2014-04-26 00:00:00
...,...,...,...,...,...,...,...
688,250075723,Xylona,Preis,Italy,female,52.0,2013-04-27 00:00:00
689,142739362,Yana,Sorensen,Germany,female,56.0,2011-10-21 00:00:00
690,374315300,Yoseph,Carroll,United Kingdom,male,55.0,2011-03-16 00:00:00


In [13]:
df_customer_list2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693 entries, 0 to 692
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer_id             693 non-null    int64  
 1   Name                    658 non-null    object 
 2   Surname                 658 non-null    object 
 3   country                 693 non-null    object 
 4   Gender                  693 non-null    object 
 5   Age                     691 non-null    float64
 6   Date Registered Online  693 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 43.3+ KB


In [14]:
df_customer_list2.isna().sum()

Customer_id                0
Name                      35
Surname                   35
country                    0
Gender                     0
Age                        2
Date Registered Online     0
dtype: int64

In [15]:
df_customer_list2.nunique()

Customer_id               693
Name                      292
Surname                   480
country                    12
Gender                      3
Age                        50
Date Registered Online    466
dtype: int64

In [16]:
# rows with missing values

# age can also be 0? must check age ranges... There is also an age 999

df_customer_list2[df_customer_list2.isna().sum(axis=1) >= 1]

,Customer_id,Name,Surname,country,Gender,Age,Date Registered Online
index,,,,,,,
9,346199714,None,None,United Kingdom,male,36.0,2011-08-04 00:00:00
79,283858641,None,None,United Kingdom,male,42.0,2011-12-25 00:00:00
89,304822153,None,None,Netherlands,male,55.0,2011-04-20 00:00:00
91,303834481,None,None,Sweden,female,57.0,2011-05-10 00:00:00
94,341075326,None,None,United Kingdom,male,66.0,2011-06-17 00:00:00
113,245230286,None,None,United Kingdom,male,42.0,2011-10-23 00:00:00
120,131513287,None,None,United Kingdom,female,33.0,2011-04-11 00:00:00
138,197622646,None,None,United Kingdom,female,51.0,2011-03-16 00:00:00
169,308555312,None,None,Spain,female,34.0,2013-05-13 00:00:00


In [17]:
# max age = 999, min age = 0, and also missing age values 

df_customer_list2.describe(include='all')

,Customer_id,Name,Surname,country,Gender,Age,Date Registered Online
count,6.930000e+02,658,658,693,693,691.000000,693
unique,NaN,292,480,12,3,NaN,466
top,NaN,Michael,Carroll,United Kingdom,male,NaN,2011-06-13 00:00:00
freq,NaN,9,5,409,411,NaN,7
mean,2.498481e+08,NaN,NaN,NaN,NaN,50.771346,NaN
std,8.526650e+07,NaN,NaN,NaN,NaN,37.201953,NaN
min,1.002696e+08,NaN,NaN,NaN,NaN,0.000000,NaN
25%,1.782263e+08,NaN,NaN,NaN,NaN,45.000000,NaN
50%,2.475545e+08,NaN,NaN,NaN,NaN,51.000000,NaN
75%,3.240252e+08,NaN,NaN,NaN,NaN,55.000000,NaN


In [18]:
# 4117 orders

# no customer id, only customer name and the name is first name + last name

order_query = """SELECT * FROM ListOfOrders"""

df_orders = pd.read_sql(order_query, sqlite_connection, index_col='index')
df_orders

,Order ID,Oder_date_time,Customer Name
index,,,
0,IT-2014-3675672,2014-3-11 10:17:42,Aaron Bergman
1,IT-2014-1559800,2014-2-7 23:0:48,Aaron Bergman
2,ES-2012-4291867,2012-3-12 18:17:46,Aaron Bergman
3,ES-2014-3856656,2014-4-23 21:48:53,Aaron Bergman
4,ES-2011-4184901,2011-8-30 15:7:28,Aaron Bergman
...,...,...,...
4112,ES-2011-4052630,2011-2-21 1:48:37,Zuschuss Donatelli
4113,IT-2012-2197138,2012-7-13 7:22:40,Zuschuss Donatelli
4114,IT-2012-4926272,2012-8-17 2:39:31,Zuschuss Donatelli


In [19]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4117 entries, 0 to 4116
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Order ID        4117 non-null   object
 1   Oder_date_time  4117 non-null   object
 2   Customer Name   4117 non-null   object
dtypes: object(3)
memory usage: 128.7+ KB


In [20]:
# no missing values

df_orders.isna().sum()

Order ID          0
Oder_date_time    0
Customer Name     0
dtype: int64

In [21]:
# all 4417 are unique order_ids

df_orders.nunique()

Order ID          4117
Oder_date_time    4117
Customer Name      792
dtype: int64

In [22]:
# rows with missing values (there are no missing values here in any rows)

df_orders[df_orders.isna().sum(axis=1) >= 1]

,Order ID,Oder_date_time,Customer Name
index,,,


In [23]:
# 8047 order breakdown items

# no order item id

# there are two rows with Total revenue before discount have value 'unknown', so those get first replaced by np.nan

order_breakdown_query = """SELECT * FROM OrderBreakdown"""

df_order_breakdown = pd.read_sql(order_breakdown_query, sqlite_connection, index_col='index')
df_order_breakdown['Total revenue before discount'] = df_order_breakdown['Total revenue before discount'].replace('unknown', np.nan).astype(float)
df_order_breakdown

,Order ID,Product Name,Discount per product,Total revenue before discount,Total quantity,Category
index,,,,,,
0,ES-2011-1001989,"Cardinal 3-Hole Punch, Clear",3.0,229.0,9.0,Office Supplies
1,ES-2011-1010958,"Cardinal 3-Hole Punch, Economy",0.0,58.0,2.0,Office Supplies
2,ES-2011-1010958,"Cameo Peel and Seal, Security-Tint",0.0,140.0,6.0,Office Supplies
3,ES-2011-1012469,"Cardinal Binding Machine, Clear",0.0,148.0,3.0,Office Supplies
4,ES-2011-1043483,"Xerox Message Books, Premium",0.0,224.0,11.0,Office Supplies
...,...,...,...,...,...,...
8042,IT-2014-5975833,"Hoover Microwave, Black",3.0,832.0,3.0,Office Supplies
8043,IT-2014-5984498,"Sauder 3-Shelf Cabinet, Traditional",15.0,83.0,1.0,Furniture
8044,IT-2014-5989338,"Eldon Lockers, Blue",3.0,1603.0,9.0,Office Supplies


In [24]:
df_order_breakdown.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8047 entries, 0 to 8046
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Order ID                       8047 non-null   object 
 1   Product Name                   8047 non-null   object 
 2   Discount per product           8047 non-null   float64
 3   Total revenue before discount  8043 non-null   float64
 4   Total quantity                 8045 non-null   float64
 5   Category                       8047 non-null   object 
dtypes: float64(3), object(3)
memory usage: 440.1+ KB


In [25]:
# no missing values for order id

df_order_breakdown.isna().sum()

Order ID                         0
Product Name                     0
Discount per product             0
Total revenue before discount    4
Total quantity                   2
Category                         0
dtype: int64

In [26]:
# rows with missing values (there are no missing values here in any rows)

df_order_breakdown[df_order_breakdown.isna().sum(axis=1) >= 1]

,Order ID,Product Name,Discount per product,Total revenue before discount,Total quantity,Category
index,,,,,,
4038,ES-2013-5430201,"Ames Clasp Envelope, with clear poly window",0.0,NaN,3.0,Office Supplies
4718,ES-2014-2353233,"Smead Legal Exhibit Labels, Laser Printer Comp...",3.0,30.0,NaN,Office Supplies
5550,ES-2014-4357731,"Acme Scissors, Serrated",0.0,111.0,NaN,Office Supplies
5987,ES-2014-5500680,"Rogers Box, Single Width",0.1,NaN,1.0,Office Supplies
6033,ES-2014-5605089,"Sanford Highlighters, Easy-Erase",0.0,NaN,2.0,Office Supplies
6568,IT-2011-5348771,"Eldon Shelving, Single Width",0.1,NaN,9.0,Office Supplies


In [27]:
df_order_breakdown.describe(include='all')

,Order ID,Product Name,Discount per product,Total revenue before discount,Total quantity,Category
count,8047,8047,8047.000000,8043.000000,8045.000000,8047
unique,4117,1810,NaN,NaN,NaN,3
top,ES-2014-5605089,"Eldon File Cart, Single Width",NaN,NaN,NaN,Office Supplies
freq,11,26,NaN,NaN,NaN,5286
mean,NaN,NaN,2.577321,291.931369,3.772032,NaN
std,NaN,NaN,4.611065,485.304890,2.203584,NaN
min,NaN,NaN,0.000000,3.000000,1.000000,NaN
25%,NaN,NaN,0.000000,49.000000,2.000000,NaN
50%,NaN,NaN,0.000000,117.000000,3.000000,NaN
75%,NaN,NaN,3.000000,313.000000,5.000000,NaN
